In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [3]:
from sklearn.metrics import mean_squared_error as mse

In [2]:
df = pd.read_csv("mobiles.csv")
df.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39


### Q1.

In [ ]:
stat_mean = df["sales"].mean()
stat_std  = df["sales"].std()
stat_out  = stat_mean + 2 * stat_std
stat_out

In [6]:
df_q1 = df.loc[df["sales"] > stat_out, ].reset_index(drop = True)
df_q1.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Medium,128,6,2,1,4000,4.6,122001,18999,0.09,231.79
1,Large,128,6,4,2,4500,4.5,267028,15999,0.20,427.22


In [ ]:
len(df_q1)

In [9]:
df_q1["idx"] = (df_q1["ROM"] / 32) + (df_q1["RAM"] / 2) + \
(df_q1["num_front_camera"] + df_q1["num_rear_camera"]) + \
(df_q1["battery_capacity"] / 1000)

In [10]:
df_q1.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales,idx
0,Medium,128,6,2,1,4000,4.6,122001,18999,0.09,231.79,14.0
1,Large,128,6,4,2,4500,4.5,267028,15999,0.20,427.22,17.5


In [11]:
round(df_q1["idx"].mean(), 2)

11.01

### Q2.

In [13]:
df_q2 = df.loc[df["num_rear_camera"] != 1, "battery_capacity":]
df_q2.head(2)

,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
1,2815,4.5,244,57149,0.04,1.39
4,2815,4.6,745,69149,0.02,5.15


In [15]:
df_q2.corr().abs().round(2)

,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
battery_capacity,1.00,0.42,0.03,0.50,0.26,0.03
ratings,0.42,1.00,0.19,0.15,0.12,0.23
num_of_ratings,0.03,0.19,1.00,0.26,0.21,0.95
sales_price,0.50,0.15,0.26,1.00,0.10,0.25
discount_percent,0.26,0.12,0.21,0.10,1.00,0.22
sales,0.03,0.23,0.95,0.25,0.22,1.00


In [16]:
df_q2.corr().abs().round(2)["sales"]

battery_capacity    0.03
ratings             0.23
num_of_ratings      0.95
sales_price         0.25
discount_percent    0.22
sales               1.00
Name: sales, dtype: float64

### Q3.

In [18]:
df_q3 = df.copy()

In [19]:
df_q3.head(1)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52


In [21]:
# df_q3_dum = pd.get_dummies(df_q3, columns = ["screen_size"]) # 시험 버전
df_q3_dum = pd.get_dummies(df_q3, dtype = "int") # Pandas 2.0.0 이상
df_q3_dum.head(2)

,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,64,2,1,1,1800,4.5,38645,32999,0.17,127.52,0,0,0,0,1
1,64,4,2,1,2815,4.5,244,57149,0.04,1.39,0,0,1,0,0


In [22]:
df_q3_dum.shape

(430, 15)

In [23]:
df_q3_dum = df_q3_dum.set_index("sales").reset_index()
df_q3_dum.head(2)

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,127.52,64,2,1,1,1800,4.5,38645,32999,0.17,0,0,0,0,1
1,1.39,64,4,2,1,2815,4.5,244,57149,0.04,0,0,1,0,0


In [25]:
df_train, df_test = train_test_split(df_q3_dum, train_size = 0.8,
                                     random_state = 123)
len(df_train), len(df_test)

(344, 86)

In [28]:
model_nor = MinMaxScaler().fit(df_train)
arr_train_nor = model_nor.transform(df_train)
arr_test_nor  = model_nor.transform(df_test)

In [29]:
arr_train_nor[:1, ]

array([[0.00394753, 0.04761905, 0.18181818, 0.33333333, 0.        ,
        0.42307692, 0.625     , 0.00396262, 0.02607261, 0.09302326,
        0.        , 0.        , 1.        , 0.        , 0.        ]])

In [35]:
ls_k = [3, 5, 7, 9, 11]
k = ls_k[0]

model_knn = KNeighborsRegressor(n_neighbors = k)
model_knn.fit(X = arr_train_nor[:, 1:],
              y = arr_train_nor[:,  0])
pred = model_knn.predict(arr_test_nor[:, 1:])
val_rmse = mean_squared_error(y_true = arr_test_nor[:, 0],
                              y_pred = pred) ** 0.5
val_rmse

0.08186677375964535

In [ ]:
# np.sqrt()
# ** 0.5

In [36]:
ls_k = [3, 5, 7, 9, 11]

ls_rmse = []
for k in ls_k:
    model_knn = KNeighborsRegressor(n_neighbors = k)
    model_knn.fit(X = arr_train_nor[:, 1:],
                  y = arr_train_nor[:,  0])
    pred = model_knn.predict(arr_test_nor[:, 1:])
    val_rmse = mean_squared_error(y_true = arr_test_nor[:, 0],
                                  y_pred = pred) ** 0.5
    ls_rmse = ls_rmse + [val_rmse]

ls_rmse

[0.08186677375964535,
 0.09879109824384892,
 0.107669855645971,
 0.11232111394853059,
 0.1136902366621185]

In [38]:
from tqdm.notebook import tqdm
import time

ls_k = [3, 5, 7, 9, 11]

ls_rmse = []
for k in tqdm(ls_k):
    time.sleep(1)
    model_knn = KNeighborsRegressor(n_neighbors = k)
    model_knn.fit(X = arr_train_nor[:, 1:],
                  y = arr_train_nor[:,  0])
    pred = model_knn.predict(arr_test_nor[:, 1:])
    val_rmse = mean_squared_error(y_true = arr_test_nor[:, 0],
                                  y_pred = pred) ** 0.5
    ls_rmse = ls_rmse + [val_rmse]

ls_rmse

  0%|          | 0/5 [00:00<?, ?it/s]

[0.08186677375964535,
 0.09879109824384892,
 0.107669855645971,
 0.11232111394853059,
 0.1136902366621185]

In [52]:
k

11

In [53]:
# pd.Series(ls_rmse, index = ls_k)
best_k =  pd.Series(ls_rmse, index = ls_k).idxmin()
best_k

3

### Q3. 추가 지시사항
다음은 저번달에 신규 출시된 경쟁사의 스마트폰 정보이다. 해당 스마트폰의 판매지수는 얼마로 예상되는가?  
※ 정규화 되지 않은 값으로 반올림하여 소수점 첫째 자리까지 출력하시오  
※ KNN 모델을 사용하며 이웃 개수는 직전에 최적이라고 판단한 k값을 사용하시오.  
* ROM: 256
* RAM: 6
* num_rear_camera: 4
* num_front_camera: 1
* battery_capacity: 4000
* ratings: 4.3
* num_of_ratings: 25000
* sales_price: 85000
* discount_percent: 0.05
* screen_size: "Large"

In [46]:
# df_t1 = pd.DataFrame(dict(ROM = 256, ~~~))
df_t1 = df_test.head(1).reset_index(drop = True)
df_t1["RAM"] = 6
df_t1["num_rear_camera"] = 4
df_t1["battery_capacity"] = 4000
df_t1["ratings"] = 4.3
df_t1["num_of_ratings"] = 25000
df_t1["sales_price"] = 85000
df_t1["discount_percent"] = 0.05
df_t1["screen_size_Large"] = 1
df_t1["screen_size_Medium"] = 0
# df_t1.transpose()
df_t1

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,5.9,256,6,4,1,4000,4.3,25000,85000,0.05,1,0,0,0,0


In [51]:
arr_t1_nor = model_nor.transform(df_t1)
arr_t1_nor

array([[0.0119438 , 0.49206349, 0.45454545, 1.        , 0.        ,
        0.42307692, 0.625     , 0.05308122, 0.51815842, 0.09302326,
        1.        , 0.        , 0.        , 0.        , 0.        ]])

In [54]:
model_knn_best = KNeighborsRegressor(n_neighbors = best_k)
model_knn_best.fit(X = arr_train_nor[:, 1:],
                   y = arr_train_nor[:,  0])
pred_t1 = model_knn_best.predict(arr_t1_nor[:, 1:])
pred_t1

array([0.00132259])

In [17]:
df_corr_melt = df_q2.corr().reset_index().melt(id_vars = "index")
df_corr_melt.head(2)

,index,variable,value
0,battery_capacity,battery_capacity,1.000000
1,ratings,battery_capacity,-0.424129


In [ ]:
arr_t1_nor[0, 0] = pred_t1
arr_t1_inv = model_nor.inverse_transform(arr_t1_nor)
arr_t1_inv

In [57]:
df_t1_inv = pd.DataFrame(arr_t1_inv, columns = df_t1.columns)
df_t1_inv

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,0.653333,256.0,6.0,4.0,1.0,4000.0,4.3,25000.0,85000.0,0.05,1.0,0.0,0.0,0.0,0.0


In [58]:
df_t1

,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,5.9,256,6,4,1,4000,4.3,25000,85000,0.05,1,0,0,0,0


In [59]:
df_t1_inv["sales"]

0    0.653333
Name: sales, dtype: float64